Ideas 1:
1. first we need to convert our sparse matrix to dense matrix where each row is an example by keeping only the diagonals.
2. Change in Shapes:
   1. for 1D models: 
      1. the shape will change from (nxn) to (nx3) since each cell has 3 terms (i.e. 2 neighbors and the cell itself). 
      2. Formulation:
         1.  b, c1, c2 x x1 > b1
         2. c1, c2, c3 x x2 > b2
         3. c2, c3, c4 x x3 > b3
         4. c3, c4, b  x x4 > b4
   2. for 2D models: 
      1. the shape will change from (nxn) to (nx5) since each cell has 5 terms (i.e. 4 neighbors and the cell itself).  
   3. for 3D models:
      1. the shape will change from (nxn) to (nx7) since each cell has 7 terms (i.e. 6 neighbors and the cell itself).

Idea 2: Train neural network to calc A^-1 then use that to get x: A^-1.b = x

In [14]:
import openresim as ors
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt

In [15]:
def create_model():
    grid = ors.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = ors.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = ors.models.Model(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model

model = create_model()

In [16]:
[print(f"cell {k}: {dict(v[0])}") for k, v in model.get_cells_eq().items()]

cell 1: {p1: 85.2012000000000, p2: -28.4004000000000}
cell 2: {p1: 28.4004000000000, p3: 28.4004000000000, p2: -56.8008000000000}
cell 3: {p2: 28.4004000000000, p4: 28.4004000000000, p3: -56.8008000000000}
cell 4: {p3: 28.4004000000000, p4: -28.4004000000000}


[None, None, None, None]

In [17]:
A, d = model.get_matrices_vectorized(sparse=False)

In [18]:
model.pressures

array([[4000.,   nan,   nan,   nan,   nan,   nan]])

In [19]:
model.run(
        nsteps=3,
        sparse=False,
        threading=False,
        vectorize=False,
        check_MB=True,
        print_arrays=True,
        isolver="cgs")

[info] Simulation run started: 3 timesteps.


 33%|███▎      | 1/3 [00:00<00:00,  6.57steps/s]

step: 0
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]



 67%|██████▋   | 2/3 [00:00<00:00,  6.57steps/s]

step: 1
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]



100%|██████████| 3/3 [00:00<00:00,  6.68steps/s]

step: 2
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]

[info] Simulation run of 3 steps finished in 0.45 seconds.

[info] Material Balance Error: 1.693933882052079e-11.


### Prepare Data

In [7]:
model.get_data(
    boundary=False,
    units=False,
    drop_nan=True,
    # drop_zero=True,
               )

,steps,Time,cells_id,pressure
0,1,1,1,3989.436768
1,2,2,1,3989.436768
2,3,3,1,3989.436768
3,1,1,2,3968.310305
4,2,2,2,3968.310305
5,3,3,2,3968.310305
6,1,1,3,3947.183842
7,2,2,3,3947.183842
8,3,3,3,3947.183842
9,1,1,4,3926.057379


In [ ]:
cells_id = model.grid.get_cells_id(False, False, "array")
cells_id_nsteps = np.array([cells_id]*model.nsteps).reshape(-1,1)
cells_nsteps = np.repeat(np.arange(1, model.nsteps+1), model.n).reshape(-1,1)
x = np.concatenate([cells_nsteps, cells_id_nsteps], axis=1)
y = model.pressures[1:, cells_id].reshape(-1,1)
data = np.concatenate([x,y], axis=1)
data

In [ ]:
model.pressures.flatten()

In [ ]:
A

In [ ]:
d

In [ ]:
np.concatenate([
    np.concatenate([[0], np.diag(A, -1)]).reshape(-1,1),
    np.diag(A, 0).reshape(-1,1),
    np.concatenate([np.diag(A, 1), [0]]).reshape(-1,1)],
    axis=1
)
df = pd.DataFrame()
df['L'] = np.concatenate([[0], np.diag(A, -1)])
df['C'] = np.diag(A, 0)
df['R'] = np.concatenate([np.diag(A, 1), [0]])
df['B'] = -d
# df['P_t0'] = model.pressures[0, 1:-1]
# df['P_t1'] = model.pressures[1, 1:-1]
df['P_L'] = model.pressures[1, 0:-2]
df['P_C'] = model.pressures[1, 1:-1]
df['P_R'] = model.pressures[1, 2:]
df.replace(np.nan, 0, inplace=True)

# df.iloc[0, :2] = [56.8008, 28.4004]

df

In [ ]:
# df = pd.DataFrame(columns=['id', 'tstep', 'p'])
df = pd.DataFrame(columns=['id'])
n = model.grid.get_n(False)
id = pd.Series(model.grid.cells_id, name='id')
print(id)
tstep = range(model.tstep)
for t in tstep:
    df_t = pd.DataFrame([id, [t]*n, model.pressures[:, id].flatten()], 
                        columns=df.columns)
    print((n*t),n+(n*t))
    df.loc[(n*t):n+(n*t)+1, 'id'] = id#, t, model.pressures[:, id].flatten()]
    # df.append(id)
df

In [ ]:
class ConstantValueConstraint(tf.keras.constraints.Constraint):
  """Constrains the elements of the tensor to `value`.
  https://stackoverflow.com/questions/65484696/set-only-the-bias-to-be-non-trainable-in-tensorflow-keras
  """

  def __init__(self, value):
    self.value = value

  def __call__(self, w):
    return w * 0 + self.value

  def get_config(self):
    return {'value': self.value}
  
from keras.constraints import nonneg

import keras.backend as K

class Between(tf.keras.constraints.Constraint):
  """_summary_

  https://stackoverflow.com/a/56822471/11549398
  """
  def __init__(self, min_value, max_value):
      self.min_value = min_value
      self.max_value = max_value

  def __call__(self, w):        
      return K.clip(w, self.min_value, self.max_value)

  def get_config(self):
      return {'min_value': self.min_value,
              'max_value': self.max_value}

In [ ]:
# example for 1D:
r = 1
bias_weights = df.loc[r, 'B']
# kernel_weights = df.loc[r, ['L','C','R']].values.transpose()
kernel_weights = df.loc[r, ['P_L','P_C','P_R']].values.transpose()
x0 = keras.Input(shape=(3,), name='input')
x = keras.layers.Dense(1,   
        use_bias=True,
        bias_initializer=tf.keras.initializers.Constant(bias_weights), 
        bias_constraint=ConstantValueConstraint(bias_weights),
        kernel_initializer=tf.keras.initializers.Constant(kernel_weights), 
        kernel_constraint=Between(min_value=0.0, max_value=4000),
        name='layer_1')(x0)
nn_model = keras.Model(inputs=x0, outputs=x)
print('Weights: ')
print(nn_model.weights)

print('Result: should be zero!')
nn_model.predict(df.loc[r, ['L','C','R']].values[np.newaxis])
# nn_model.predict(df.loc[r, ['P_L','P_C','P_R']].values[np.newaxis])


In [ ]:
# example for 1D Conv:
r = 0
bias_weights = df.loc[r, 'B']
# kernel_weights = df.loc[:, ['L','C','R']].values
kernel_weights = df.loc[r, ['P_L','P_C','P_R']].values

x0 = keras.Input(shape=(3,1), name='input')
x = keras.layers.Conv1D(1, 
                        kernel_size=(3,),
                        strides=1,
                        kernel_initializer=tf.keras.initializers.Constant(kernel_weights),
                        # bias_initializer=tf.keras.initializers.Constant(bias_weights),
                        # bias_constraint=ConstantValueConstraint(bias_weights),
                        kernel_constraint=Between(min_value=0.0, max_value=4000),
                        use_bias=True)(x0)
# x = keras.layers.Dense(1, 
#                     use_bias=False,
#                     kernel_initializer=tf.keras.initializers.Constant(1))(x)
nn_model = keras.Model(inputs=x0, outputs=x)
print('Weights: ')
print(nn_model.weights)

print('Result: should be zero!')
nn_model.predict(df.loc[r, ['L','C','R']].values.flatten()[np.newaxis])

opt = keras.optimizers.Adam(learning_rate=100)
nn_model.compile(opt, 'MSE')
nn_model.fit(
    df.loc[:, ['L','C','R']].values, 
    df.loc[:, ['B']].values,
    # df.loc[:, ['P_C']].values.flatten()[np.newaxis],
    batch_size=1,
    epochs=100,
    shuffle=False
)


In [ ]:
nn_model.weights

In [ ]:
print('Result: should be zero!')
nn_model.predict(df.iloc[:, 0:3].values)

In [ ]:
r = 4
bias = np.array(df.loc[:r, 'B'])
x0 = keras.Input(shape=(3,1), name='input')
x = keras.layers.Conv1D(3, 
                        kernel_size=(1,),
                        use_bias=False)(x0)
x = keras.layers.Dense(1, 
                    use_bias=False)(x)
# x = keras.layers.Dense(4,   
#         use_bias=True,
#         bias_initializer=tf.keras.initializers.Constant(df.loc[:r, 'B']), 
#         bias_constraint=ConstantValueConstraint(df.loc[:r, 'B']),
#         kernel_constraint=nonneg(),
#         name='layer_1')(x)
nn_model = keras.Model(inputs=x0, outputs=x)
nn_model.weights
# x2 = keras.layers.Dense(1, 
#         # use_bias=True,
#         # kernel_constraint=nonneg(),
#         name='layer_2')(x1)
# nn_model = keras.Model(inputs=x0, outputs=x2)
# nn_model.weights

In [ ]:
nn_model.get_layer('layer_1').set_weights([
    # nn_model.get_layer('layer_1').get_weights()[0],
    # np.array([4000,3999,3998]).reshape(-1,1),
    # df.loc[:r, ['L','C','R']].values.transpose(),
    np.array(df.loc[:r, 'B']),
])
nn_model.get_layer('layer_2').set_weights([
    np.array(df.loc[:r, 'B']).reshape(-1, 1),
    # np.array([4000,3999,3998]).reshape(-1,1),
    # df.loc[r, ['L','C','R']].values.reshape(-1,1),
    np.array([1]),
])
nn_model.weights

### Using Autoencoder

In [ ]:
Ad = np.concatenate([A,d], axis=1)
Ad

In [ ]:
ACT_LST = [None, 'linear', 'sigmoid', 'relu', 'tanh']
FLT_LST = [5, 10, 20, 20, 5, 1]
IN_SHAPE = Ad.shape[1]

def AutoEncoder(
    input_shape = (IN_SHAPE,),
    filters= FLT_LST,
    activation_function = ACT_LST[-1], 
    dropout = False,
    dropout_rate = 0.2,
):
    """Create keras autoencoder.

    Parameters
    ----------
    input_shape : Tuple[int], optional
        _description_, by default (X.shape[1],)
    filters : List[int], optional
        _description_, by default [64, 32, 16, 3]
    activation_function : str, optional
        _description_, by default ACT_LST[2]
    dropout : bool, optional
        _description_, by default False
    dropout_rate : float, optional
        _description_, by default 0.1

    Returns
    -------
    _type_
        _description_
    """


    input = keras.Input(shape=input_shape)
    for i, filter in enumerate(filters):
        if i == 0:
            x = tf.keras.layers.Dense(filter, activation=activation_function)(input)
            if dropout:
                x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
        elif i < len(filters) - 1:
            x = tf.keras.layers.Dense(filter, activation=activation_function)(x)
            if dropout:
                x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
        else:
            compressed = tf.keras.layers.Dense(filter, activation=activation_function)(x)
    for i, filter in enumerate(filters[:-1][::-1]):
        if i == 0:
            x = tf.keras.layers.Dense(filter, activation=activation_function)(compressed)
            if dropout:
                x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
        else:
            x = tf.keras.layers.Dense(filter, activation=activation_function)(x)
            if dropout:
                x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
        decompressed = tf.keras.layers.Dense(input_shape[0], activation=activation_function)(x)

    autoencoder = keras.Model(input, decompressed, name='autoencoder')
    encoder = keras.Model(input, compressed, name='encoder')

    compressed_input = keras.Input(shape=(filters[-1],))
    bottleneck_id = -(len(filters)*2)+1 if dropout else -(len(filters))
    for i, layer in enumerate(autoencoder.layers[bottleneck_id:]):
        if i == 0:
            decoder_layer = layer(compressed_input)
        else:
            decoder_layer = layer(decoder_layer)
    decoder = keras.Model(compressed_input, decoder_layer, name='decoder')
    # Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl
    opt = keras.optimizers.Adam(learning_rate=10)
    encoder.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    decoder.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    autoencoder.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return autoencoder, encoder, decoder

autoencoder, encoder, decoder = AutoEncoder()
autoencoder.summary()

In [ ]:
import time

epochs = 300
eval_freq = 50
results = {'train_loss':[], 'train_mse':[],
               'test_loss':[], 'test_mse':[],
               'n_way_train_acc':[], 'n_way_test_acc':[]}

start_time = time.time()

for epoch in range(epochs+1):
    outputs = encoder.fit(Ad, model.pressures[-1, 1:-1],
                epochs=1,
                batch_size=5,
                shuffle=False,
                verbose=0,
                validation_data=(Ad, model.pressures[-1, 1:-1]))
    
    if epoch % eval_freq == 0:
        results['train_loss'].append(outputs.history['loss'][0])
        results['train_mse'].append(outputs.history['mean_squared_error'][0])
        results['test_loss'].append(outputs.history['val_loss'][0])
        results['test_mse'].append(outputs.history['val_mean_squared_error'][0])
        train_loss = round(results['train_loss'][-1],3)
        test_loss = round(results['test_loss'][-1],3)
        
        print("-"*80)
        process_time = round((time.time()-start_time) / 60.0, 1)
        print(f"[{process_time} mins] epoch: {epoch} | train loss:{train_loss} | test loss: {test_loss}")
        
plt.plot(results['train_loss'], label='train_loss')
plt.plot(results['test_loss'], label='test_loss')
plt.legend()
plt.show();

### Thesis

In [ ]:
model.pressures[1, 1:-1]

In [ ]:
m = np.zeros((4,4))
np.fill_diagonal(m, 0.0001)
x = np.array([0, 0, 0, 0])
energy_function = 0
lstsq_energies = []
for i in range(10000):
    error_vector = A @ x - d
    energy_function = (1/2) * error_vector.T @ error_vector
    lstsq_energies.append(energy_function)
    gradient_vector = A.T @ error_vector
    scaled_gradient = -m @ gradient_vector
    x_new = x + scaled_gradient
    x = x_new

np.concatenate([x[:,0],model.pressures[1, 1:-1]])

In [ ]:
lstsq_loss_without_tf = lstsq_energies[-1]
print("Loss without using tensorflow is ", lstsq_loss_without_tf)

In [ ]:
b = d
n_dim = 4
n_training_steps_per_epoch = 1
x_train = np.zeros((n_training_steps_per_epoch,n_dim))
NET_LEARNING_RATE = 1e-2
NET_EPOCHS = 1000
NET_BATCH_SIZE = 1

class LinearSystemSolution(tf.keras.Model):
    def __init__(self, n_input_dimension, A, b, **kwargs):
        super(LinearSystemSolution, self).__init__(**kwargs)
        self.solution = tf.Variable(initial_value=tf.zeros((
        n_input_dimension,1), dtype=tf.float64),
        trainable= True ,
        dtype=tf.float64,
        shape=(n_input_dimension,1))
        self.A = tf.convert_to_tensor(A)
        self.b = tf.convert_to_tensor(b)
    def call(self, x, training=True):
        error = tf.matmul(self.A, self.solution) - self.b
        self.add_loss(tf.reduce_sum(tf.square(error))/2)
        return self.solution
    
network = LinearSystemSolution(n_dim, A, b)
network.compile(optimizer=tf.optimizers.SGD(learning_rate=NET_LEARNING_RATE))
# callbacks = [LossAndErrorPrintingCallback()]
lstsq_historian = network.fit(x_train, 
                              epochs=NET_EPOCHS,
                              verbose=1, 
                              batch_size=NET_BATCH_SIZE, 
                            #   callbacks=callbacks
                              )
lstsq_energies[999]

In [ ]:
lstsq_loss_with_tf = (lstsq_historian.history["loss"][-1])
print("Loss using tensorflow is ", lstsq_loss_with_tf)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.semilogy(lstsq_historian.history["loss"], "x",
label = "with tf", markevery= 5)
ax.semilogy(lstsq_energies, label = "without tf")
ax.set_title("Loss: Ordinary Least Squares Problem")
ax.set_xlabel("Iterations")
ax.set_ylabel("Loss")
ax.legend()